# This notebook provides the commands needed to read in an NWB file with raw data taken from a trodes .rec file and to copy the relevant metadata into a DataJoint Schema designed to mimic NWB structure

# The notebook then filters the LFP data based on 

## The following steps must be completed before running this notebook:
### 1) Set up access to a DataJoint database. See https://tutorials.datajoint.io/
### 2) Install pynwb. 
    See https://pynwb.readthedocs.io/en/stable/getting_started.html#installation
### 3) Install the ndx-fl-novela package. 
     conda install -c novelakrk ndx-fl-novela
### 4) Download the example franklab NWB file from DropBox. 
    https://www.dropbox.com/to_be_updated

#### Make sure we're in the franklabnwb/franklabnwb directory. 
#### Eventually this will be unnecessary because the package will properly installed

#### Load all of the relevant modules

In [1]:
%env DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE
%load_ext autoreload
%autoreload 2


import pynwb
import os

#DataJoint and DataJoint schema
import datajoint as dj

dj.config["enable_python_native_blobs"] = True
dj.config['stores'] = {
  'local': dict(  # 'regular' external storage for this pipeline
                protocol='file',
                location='/data/nwb_builder_test_data/dj/',
                stage='/data/nwb_builder_test_data/dj/'),
  'local-filtered': dict( # 'local_filtered' storage filtered data in hdf5 files
                protocol='file',
                location='/data/nwb_builder_test_data/dj/filtered/',
                stage='/data/nwb_builder_test_data/dj/filtered/')
}

from nwb_datajoint import *
import ndx_franklab_novela.probe


env: DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE


Please enter DataJoint username:  root
Please enter DataJoint password:  ········


Connecting root@localhost:3306


09:51:35 [I] klustakwik KlustaKwik2 version 0.2.6


#### Next we select the NWB file, which corresponds to the dataset we want to extract LFP from

In [4]:
nwb_file_names = common_session.Nwbfile().fetch('nwb_file_name')
# take the first one for this demonstration
nwb_file_name = nwb_file_names[0]

#### Create the standard LFP Filters. This only needs to be done once.

In [5]:
common_filter.create_standard_filters()

### Select all electrodes for LFP

In [8]:
electrode_ids = common_ephys.ElectrodeConfig.Electrode.fetch('electrode_id')
common_ephys.LFPElectrode().set_lfp_elect(nwb_file_name, electrode_ids.tolist())


About to delete:
Nothing to delete


### Or select one electrode for LFP


In [6]:
common_ephys.LFPElectrode().set_lfp_elect(nwb_file_name, [2])


About to delete:
Nothing to delete


### populate the LFP table. Note that this takes a few hours on a laptop if you use all electrodes

In [9]:
import time
tic = time.perf_counter()
common_ephys.LFP().populate()
toc = time.perf_counter()

Output array should have shape (1091950, 192) and dtype <f8
Output array should have shape (212812, 192) and dtype <f8
Output array should have shape (2557744, 192) and dtype <f8
Output array should have shape (1356939, 192) and dtype <f8
Output array should have shape (1680945, 192) and dtype <f8
Checking output array shape is disabled, make sure portion of output array has shape (1091950, 192)
Checking output array shape is disabled, make sure portion of output array has shape (212812, 192)
Checking output array shape is disabled, make sure portion of output array has shape (2557744, 192)
Checking output array shape is disabled, make sure portion of output array has shape (1356939, 192)
Checking output array shape is disabled, make sure portion of output array has shape (1680945, 192)


/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/datajoint/connection.py:218: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


LostConnectionError: Connection was lost during a transaction.

In [11]:
toc = time.perf_counter()

toc - tic

18972.756563538

In [12]:
common_ephys.LFP()

nwb_file_name,electrode_id the unique number for this electrode,linked_file_name the name of each linked file,interval_name descriptive name of this interval list,filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,nwb_object_id the NWB object ID for loading this object from the linked file,"sampling_rate the sampling rate, in HZ"
/data/nwb_builder_test_data/beans20190718_pp.nwb,1,/data/nwb_builder_test_data/beans20190718_pp00000002.nwb,raw data valid times,LFP 0-400 Hz,20000,54b3a111-1fbc-4635-bd42-b5466dbf990d,20000.0


/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/datajoint/connection.py:218: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


nwb_file_name the name of the nwb file (same as nwb_file_location),linked_file_name the name of the linked file,linked_file_location the datajoint managed location of the linked file
